In [1]:
from google.colab import drive
import os

# set path to project folder
gdrive_path='/content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/' # Luca's Path
#gdrive_path='/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/' # Luis' Path

# mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# navigate to Google Drive folder
os.chdir(gdrive_path)

# check that we are in the right folder
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.git', 'README.md', 'data', 'data_loader.ipynb', 'data_loading', 'example_arrows', 'gaussian_splatting.ipynb', 'gitignore', 'gs_intro.ipynb', 'open_nerf.ipynb', 'outputs', 'visual_data.ipynb']


# Gaussian Splatting

We will use `nerfstudio` to train, run, and visualize our results. Make sure to have the following repository structure established:
```
adl4vc  
|_ nerf_segmentation  
    |_ data  
        |_ lerf  
        |_ replica  
|_ opennerf
```

Let's start by installing some requirements. Building wheels for `ninja` will take a while (~ 15min).

In [2]:
!python -m pip install --upgrade pip
!python -m pip install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
!python -m pip install ninja git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
  Cloning https://github.com/NVlabs/tiny-cuda-nn/ to /tmp/pip-req-build-pbjzza3g
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/tiny-cuda-nn/ /tmp/pip-req-build-pbjzza3g
  Resolved https://github.com/NVlabs/tiny-cuda-nn/ to commit 2ec562e853e6f482b5d09168705205f46358fb39
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.5 MB/s eta 0:00:00
  Created wheel for tinycudann: filename=tinycudann-1.7-cp310-cp310-linux_x86_64.whl size=30358232 sha256=319d02eb37d596168ad926a15675bda75a94b13fc8d57e29b4305c13079b09c8
  Stored in directory: /tmp/pip

Once, we have fulfilled these requirements, we can install nerfstudio.

In [3]:
!python -m pip install nerfstudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of imageio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of nuscenes-devkit to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jupyter-server to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

Once we have installed `nerfstudio`, we can make use of their implementation of Gaussian Splatting. It is called `splatfacto` to delineate from the original Gaussian Splatting paper. Let's start by training it scenes from Replica! With a T4-GPU, this will take ~35min for a single scene.

In [ ]:
!ns-train splatfacto --data /content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/data/nerfstudio_/replica_office0

Streaming output truncated to the last 5000 lines.
---------------------------------------------------------------------------------------------------- 
Viewer running locally at: http://localhost:7007 (listening on 0.0.0.0)                              
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
26380 (87.93%)      48.193 ms            2 m, 54 s            13.32 M                                
26390 (87.97%)      48.017 ms            2 m, 53 s            13.35 M                                
26400 (88.00%)      49.971 ms            2 m, 59 s            13.06 M                                
26410 (88.03%)      48.958 ms            2 m, 55 s            13.67 M                                
26420 (88.07%)      47.345 ms            2 m, 49 s            13.86 M                                
26430 (88.10%)      48.218 ms  

During training, we are referred to `http://localhost:7007`. In Google Colab, it is not that simple to find that where that localhost is running. Execute the following cell to find out!

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(7007)"))